In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt  
import seaborn as sns  

from sklearn.linear_model import LogisticRegression  
from sklearn import tree  
from sklearn.ensemble import RandomForestClassifier  
from sklearn.svm import SVC  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import GaussianNB  
from sklearn.cross_validation import cross_val_score  
from sklearn import metrics 

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder

feature_cols = ["teacher_prefix","school_state","project_grade_category","project_subject_categories",\
                "teacher_number_of_previously_posted_projects","project_title","project_essay_1",\
                "project_essay_2","project_essay_3","project_essay_4","project_resource_summary",\
                'project_submitted_datetime']
target_cols = ["project_is_approved"]
train=pd.read_csv("/Users/schwalmdaniel/hullaballoo_android/ml/donors_train.csv"\
                  ,usecols=['id'] + feature_cols + target_cols, converters={"project_essay_1":str,\
                    "project_essay_2":str,"project_essay_3":str,"project_essay_4":str,"project_resource_summary":str})
test=pd.read_csv("/Users/schwalmdaniel/hullaballoo_android/ml/donors_test.csv",usecols=['id'] + feature_cols,\
                    converters={"project_essay_1":str,\
                    "project_essay_2":str,"project_essay_3":str,"project_essay_4":str,"project_resource_summary":str})
resources=pd.read_csv("/Users/schwalmdaniel/hullaballoo_android/ml/resources.csv",usecols=['id','price',\
                                                                                           'quantity','description'])

resources['description'].fillna('unknown',inplace=True)


resources_price = pd.DataFrame(resources[['id', 'price']].groupby('id').price.agg(\
    [
        'count', 
        'sum', 
        'min', 
        'max', 
        'mean', 
        'std', 
        #'median',
        lambda x: len(np.unique(x)),
    ])).reset_index()
#print(resources_price.head())
resources_quantity = pd.DataFrame(resources[['id', 'quantity']].groupby('id').quantity.agg(\
    [ 
        'sum', 
        'min', 
        'max', 
        'mean', 
        'std', 
        #'median',
        lambda x: len(np.unique(x)),
    ])).reset_index()
#print(resources_quantity.head())

resources_price['mean_price'] = resources_price['sum']/resources_price['count']
resources_price['price_max_to_price_min'] = resources_price['max']/resources_price['min']
resources_quantity['quantity_max_to_quantity_min'] = resources_quantity['max']/resources_quantity['min']

resources_desc = resources.groupby(['id'])['description'].apply(' '.join).reset_index()
                              
seasons = [0,0,1,1,1,2,2,2,3,3,3,0] #dec - feb is winter, then spring, summer, fall etc

test=test.merge(resources_price,on='id', how='left')
test=test.merge(resources_quantity,on='id', how='left')
test=test.merge(resources_desc,on='id', how='left')
test['project_title_len'] = test['project_title'].apply(lambda x: len(x))
test['project_title_wc'] = test['project_title'].apply(lambda x: len(x.split()))
#test['project_title_uwc'] = test['project_title'].apply(lambda x: len(set(x.lower().split())))
#test['project_title_wc_uwc_ratio'] = test['project_title_uwc'] / test['project_title_wc']
test['project_essay_1_len'] = test['project_essay_1'].apply(lambda x: len(x))
test['project_essay_1_wc'] = test['project_essay_1'].apply(lambda x: len(x.split()))
#test['project_essay_1_uwc'] = test['project_essay_1'].apply(lambda x: len(set(x.lower().split())))
#test['project_essay_1_uwc_ratio'] = test['project_essay_1_uwc'] / test['project_essay_1_wc']
test['project_essay_2_len'] = test['project_essay_2'].apply(lambda x: len(x))
test['project_essay_2_wc'] = test['project_essay_2'].apply(lambda x: len(x.split()))
#test['project_essay_2_uwc'] = test['project_essay_2'].apply(lambda x: len(set(x.lower().split())))
#test['project_essay_2_uwc_ratio'] = test['project_essay_2_uwc'] / test['project_essay_2_wc']
test['project_essay_3_len'] = test['project_essay_3'].apply(lambda x: len(x))
test['project_essay_3_wc'] = test['project_essay_3'].apply(lambda x: len(x.split()))
#test['project_essay_3_uwc'] = test['project_essay_3'].apply(lambda x: len(set(x.lower().split())))
#test['project_essay_3_uwc_ratio'] = test['project_essay_3_uwc'] / test['project_essay_3_wc']
test['project_essay_4_len'] = test['project_essay_4'].apply(lambda x: len(x))
test['project_essay_4_wc'] = test['project_essay_4'].apply(lambda x: len(x.split()))
#test['project_essay_4_uwc'] = test['project_essay_4'].apply(lambda x: len(set(x.lower().split())))
#test['project_essay_4_uwc_ratio'] = test['project_essay_4_uwc'] / test['project_essay_4_wc']
test['project_resource_summary_len'] = test['project_resource_summary'].apply(lambda x: len(x))
test['project_resource_summary_wc'] = test['project_resource_summary'].apply(lambda x: len(x.split()))
#test['project_resource_summary_uwc'] = test['project_resource_summary'].apply(lambda x: len(set(x.lower().split())))
#test['project_resource_summary_uwc_ratio'] = test['project_resource_summary_uwc'] / test['project_resource_summary_wc']
test['description_len'] = test['description'].apply(lambda x: len(x))
test['description_wc'] = test['description'].apply(lambda x: len(x.split()))
test['previous_projects_0'] = test['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if x == 0 else 0)
test['previous_projects_1_5'] = test['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if 6 > x > 0 else 0)
test['previous_projects_6_25'] = test['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if 26 > x > 5 else 0)
test['previous_projects_26_50'] = test['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if 51 > x > 25 else 0)
test['previous_projects_50'] = test['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if  x > 50 else 0)
test['project_submitted_year'] = test['project_submitted_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year)
test['project_submitted_quarter'] = test['project_submitted_datetime'].apply(lambda x: (datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month-1)//3 + 1)
test['project_submitted_yearmonth'] = test['project_submitted_datetime'].apply(lambda x: int(str(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year) + str(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month)) )
test['project_submitted_month'] = test['project_submitted_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month)
test['project_submitted_weekday'] = test['project_submitted_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday())
test['project_submitted_hour'] = test['project_submitted_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour)
test['project_submitted_season'] = test['project_submitted_datetime'].apply(lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month-1)])
test["gender"] = None
test.loc[test['teacher_prefix'] == "Mr.","gender"] = "Male"
test.loc[test['teacher_prefix'] == "Teacher","gender"] = "Not Specified"
test.loc[(test['teacher_prefix'] == "Mrs.")|(test['teacher_prefix'] == "Ms."),"gender"] = "Female"

lbl = LabelEncoder()
test["gender"] = lbl.fit_transform(test["gender"].astype(str))
test.fillna(0, inplace=True)

#print (test['gender'].unique())

#print(test.head())

train=train.merge(resources_price,on='id', how='left')
train=train.merge(resources_quantity,on='id', how='left')
train=train.merge(resources_desc,on='id', how='left')
train['project_title_len'] = train['project_title'].apply(lambda x: len(x))
train['project_title_wc'] = train['project_title'].apply(lambda x: len(x.split()))
#train['project_title_uwc'] = train['project_title'].apply(lambda x: len(set(x.lower().split())))
#train['project_title_wc_uwc_ratio'] = train['project_title_uwc'] / train['project_title_wc']
train['project_essay_1_len'] = train['project_essay_1'].apply(lambda x: len(x))
train['project_essay_1_wc'] = train['project_essay_1'].apply(lambda x: len(x.split()))
#train['project_essay_1_uwc'] = train['project_essay_1'].apply(lambda x: len(set(x.lower().split())))
#train['project_essay_1_uwc_ratio'] = train['project_essay_1_uwc'] / train['project_essay_1_wc']
train['project_essay_2_len'] = train['project_essay_2'].apply(lambda x: len(x))
train['project_essay_2_wc'] = train['project_essay_2'].apply(lambda x: len(x.split()))
#train['project_essay_2_uwc'] = train['project_essay_2'].apply(lambda x: len(set(x.lower().split())))
#train['project_essay_2_uwc_ratio'] = train['project_essay_2_uwc'] / train['project_essay_2_wc']
train['project_essay_3_len'] = train['project_essay_3'].apply(lambda x: len(x))
train['project_essay_3_wc'] = train['project_essay_3'].apply(lambda x: len(x.split()))
#train['project_essay_3_uwc'] = train['project_essay_3'].apply(lambda x: len(set(x.lower().split())))
#train['project_essay_3_uwc_ratio'] = train['project_essay_3_uwc'] / train['project_essay_3_wc']
train['project_essay_4_len'] = train['project_essay_4'].apply(lambda x: len(x))
train['project_essay_4_wc'] = train['project_essay_4'].apply(lambda x: len(x.split()))
#train['project_essay_4_uwc'] = train['project_essay_4'].apply(lambda x: len(set(x.lower().split())))
#train['project_essay_4_uwc_ratio'] = train['project_essay_4_uwc'] / train['project_essay_4_wc']
train['project_resource_summary_len'] = train['project_resource_summary'].apply(lambda x: len(x))
train['project_resource_summary_wc'] = train['project_resource_summary'].apply(lambda x: len(x.split()))
#train['project_resource_summary_uwc'] = train['project_essay_4'].apply(lambda x: len(set(x.lower().split())))
#train['project_resource_summary_uwc_ratio'] = train['project_resource_summary_uwc'] / train['project_resource_summary_wc']
train['description_len'] = train['description'].apply(lambda x: len(x))
train['description_wc'] = train['description'].apply(lambda x: len(x.split()))
train['previous_projects_0'] = train['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if x == 0 else 0)
train['previous_projects_1_5'] = train['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if 6 > x > 0 else 0)
train['previous_projects_6_25'] = train['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if 26 > x > 5 else 0)
train['previous_projects_26_50'] = train['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if 51 > x > 25 else 0)
train['previous_projects_50'] = train['teacher_number_of_previously_posted_projects'].apply(lambda x: 1 if  x > 50 else 0)
train['project_submitted_year'] = train['project_submitted_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year)
train['project_submitted_quarter'] = train['project_submitted_datetime'].apply(lambda x: (datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month-1)//3 + 1)
train['project_submitted_yearmonth'] = train['project_submitted_datetime'].apply(lambda x: int(str(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year) + str(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month)) )
train['project_submitted_month'] = train['project_submitted_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month)
train['project_submitted_weekday'] = train['project_submitted_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday())
train['project_submitted_hour'] = train['project_submitted_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour)
train['project_submitted_season'] = train['project_submitted_datetime'].apply(lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month-1)])
train["gender"] = None
train.loc[train['teacher_prefix'] == "Mr.","gender"] = "Male"
train.loc[train['teacher_prefix'] == "Teacher","gender"] = "Not Specified"
train.loc[(train['teacher_prefix'] == "Mrs.")|(train['teacher_prefix'] == "Ms."),"gender"] = "Female"
train["gender"] = lbl.fit_transform(train["gender"].astype(str))
train.fillna(0, inplace=True)

#print(train.head())

#train.loc[train['teacher_number_of_previously_posted_projects'].idxmax()]
#train.hist(column='teacher_number_of_previously_posted_projects')


In [ ]:
mystopwords = (stopwords.words('english')) + list(stop_words.ENGLISH_STOP_WORDS) 
#count_vect = CountVectorizer(stop_words=mystopwords,analyzer='word',token_pattern='[^\W\d_]{2,}',\
#                             ngram_range=(1,1), max_features=800000, min_df=2,strip_accents='unicode')
#
texts = train['project_essay_1'].tolist() + train['project_essay_2'].tolist() \
    + train['project_essay_3'].tolist() + train['project_essay_4'].tolist() \
    + train['project_resource_summary'].tolist() + train['project_title'].tolist() + train['description'].tolist() 
    
train['project_essay'] = train.apply(lambda row: ' '.join([
    str(row['project_essay_1']), 
    str(row['project_essay_2']), 
    str(row['project_essay_3']), 
    str(row['project_essay_4']),
    ]), axis=1)
test['project_essay'] = test.apply(lambda row: ' '.join([ 
    str(row['project_essay_1']), 
    str(row['project_essay_2']), 
    str(row['project_essay_3']), 
    str(row['project_essay_4']),
    ]), axis=1)
 
#stemmer = PorterStemmer()

#train['project_essay'] = train['project_essay'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split(" ")]))
#test['project_essay'] = test['project_essay'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split(" ")]))
    
from sklearn.feature_extraction.text import TfidfVectorizer
print('Preprocessing text...')
cols = [
    'project_essay',
    'description',
    #'project_essay_2', 
    #'project_essay_3',
    #'project_essay_4',
    'project_title',
    'project_resource_summary'
]

n_features = [
    4000,
    400,
    400,
    400
]

for c_i, c in enumerate(cols):
    print ('Processing col: ' + c)
    tfidf = TfidfVectorizer(max_features=n_features[c_i], min_df=4,stop_words=mystopwords,analyzer='word',\
                            token_pattern='[^\W\d_]{2,}', ngram_range=(1,1),strip_accents='unicode')
    tfidf.fit(texts)
    
    tfidf_train = np.array(tfidf.transform(train[c]).todense(), dtype=np.float16)
    tfidf_test = np.array(tfidf.transform(test[c]).todense(), dtype=np.float16)

    for i in range(n_features[c_i]):
        train[c + '_tfidf_' + str(i)] = tfidf_train[:, i]
        test[c + '_tfidf_' + str(i)] = tfidf_test[:, i]
        
    del tfidf, tfidf_train, tfidf_test    
    
#train.head()

'''X_train_counts = count_vect.fit_transform(texts)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
weights = np.asarray(X_train_tfidf.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': count_vect.get_feature_names(), 'weight': weights})
weights_df = weights_df.sort_values(ascending=False, by=['weight'])
weights_df.head()'''

In [ ]:
print('performing sentiment analysis')

import nltk
nltk.download('vader_lexicon')
SIA = SentimentIntensityAnalyzer()

for col in cols:
    print ('analyzing col ' + col)
    train[col+"_vader_Compound"]= train[col].apply(lambda x:SIA.polarity_scores(x)['compound'])
    train[col+'_vader_Negative']= train[col].apply(lambda x:SIA.polarity_scores(x)['neg'])
    train[col+'_vader_Positive']= train[col].apply(lambda x:SIA.polarity_scores(x)['pos'])
    test[col+"_vader_Compound"]= test[col].apply(lambda x:SIA.polarity_scores(x)['compound'])
    test[col+'_vader_Negative']= test[col].apply(lambda x:SIA.polarity_scores(x)['neg'])
    test[col+'_vader_Positive']= test[col].apply(lambda x:SIA.polarity_scores(x)['pos'])
    

In [ ]:
count_vect = CountVectorizer(stop_words=mystopwords,analyzer='word',token_pattern='[^\W\d_]{2,}',\
                             ngram_range=(1,1), max_features=4000, min_df=4,strip_accents='unicode')
X_train_counts = count_vect.fit_transform(texts)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
weights = np.asarray(X_train_tfidf.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': count_vect.get_feature_names(), 'weight': weights})
weights_df = weights_df.sort_values(ascending=False, by=['weight'])
weights_df.head()
weights_df.to_csv('weights_df.csv', index=False,quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
categories = train['project_subject_categories'].unique()
subcategories = train['project_subject_subcategories'].unique()

cat_set = set()
for cat in categories:
    for c in cat.split(','):
        cat_set.add(c.strip())
#print(cat_set)
for cat in cat_set:
    train['project_subject_categories_' + cat] =  train['project_subject_categories'].apply(lambda x : 1 if cat in x else 0)
    test['project_subject_categories_' + cat] =  test['project_subject_categories'].apply(lambda x : 1 if cat in x else 0)
    
sub_cat_set = set()
for cat in subcategories:
    for c in cat.split(','):
        sub_cat_set.add(c.strip())
#print(sub_cat_set)
for cat in sub_cat_set:
    train['project_subject_subcategories_' + cat] =  train['project_subject_subcategories'].apply(lambda x : 1 if cat in x else 0)
    test['project_subject_subcategories_' + cat] =  test['project_subject_subcategories'].apply(lambda x : 1 if cat in x else 0)


In [ ]:
y = train['project_is_approved']

train['school_state'] = 'state_' + train['school_state'].astype(str)  
train['teacher_prefix'] = 'teacher_prefix_' + train['teacher_prefix'].astype(str)
train['project_grade_category'] = 'project_grade_category' + train['project_grade_category'].astype(str)
train['project_subject_categories'] = 'project_subject_categories' + train['project_subject_categories'].astype(str)
#train['project_subject_subcategories'] = 'project_subject_subcategories' + train['project_subject_subcategories'].astype(str)

train = train.join(pd.get_dummies(train['school_state']))
train = train.join(pd.get_dummies(train['teacher_prefix']))
train = train.join(pd.get_dummies(train['project_grade_category']))
train = train.join(pd.get_dummies(train['project_subject_categories']))
#train = train.join(pd.get_dummies(train['project_subject_subcategories']))

train2 = train.drop(['teacher_prefix_Mr.','state_NV', 'project_grade_categoryGrades PreK-2',\
                     'project_subject_categoriesLiteracy & Language','project_subject_subcategoriesLiteracy'], errors='ignore', axis=1)  
train2 = train2.drop(['id','teacher_prefix','school_state','project_grade_category',\
                      'project_subject_subcategories', 'project_subject_categories','project_title',"project_essay_1",\
                "project_essay_2","project_essay","project_essay_3","project_essay_4",\
                      "project_essay","project_resource_summary",\
                      'project_submitted_datetime','description','count_x','count_y'], errors='ignore',axis=1)  
train2.head()

In [ ]:
print ((train2.columns.tolist()))

In [ ]:
test['school_state'] = 'state_' + test['school_state'].astype(str)  
test['teacher_prefix'] = 'teacher_prefix_' + test['teacher_prefix'].astype(str)
test['project_grade_category'] = 'project_grade_category' + test['project_grade_category'].astype(str)
test['project_subject_categories'] = 'project_subject_categories' + test['project_subject_categories'].astype(str)
#test['project_subject_subcategories'] = 'project_subject_subcategories' + test['project_subject_subcategories'].astype(str)

test = test.join(pd.get_dummies(test['school_state']))
test = test.join(pd.get_dummies(test['teacher_prefix']))
test = test.join(pd.get_dummies(test['project_grade_category']))
test = test.join(pd.get_dummies(test['project_subject_categories']))
#test = test.join(pd.get_dummies(test['project_subject_subcategories']))

test2 = test.drop(['teacher_prefix_Mr.','state_NV', 'project_grade_categoryGrades PreK-2', \
                   'project_subject_categoriesLiteracy & Language','project_subject_subcategoriesLiteracy'], axis=1, errors='ignore')  
test2 = test2.drop(['id','teacher_prefix','school_state','project_grade_category',\
                    'project_subject_subcategories','project_subject_categories','project_title',"project_essay_1",\
                "project_essay_2","project_essay_3","project_essay_4","project_essay", "project_resource_summary",\
                'project_submitted_datetime','description','count_x','count_y'],axis=1, errors='ignore')

test2.head()

In [ ]:
#trainn, test, validate = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))]) 

#print (trainn.shape, test.shape, validate.shape)

y_train = train2['project_is_approved']  
x_train = train2.drop(['project_is_approved','teacher_prefix','school_state','project_grade_category',\
                       'project_subject_subcategories','project_subject_categories','project_title'"project_essay_1",\
                "project_essay_2","project_essay_3","project_essay_4","project_resource_summary"], errors='ignore',axis=1)  
y_test = train[target_cols]  
#x_test = test.drop(['project_is_approved','teacher_prefix','school_state','project_grade_category','project_subject_categories'], axis=1)  

#x_testt = testt[feature_cols]


'''print (test2.shape)
print (y_test.shape)

rf = RandomForestClassifier()  
rf.fit(x_train, y_train)  
print (rf.score(x_train, y_train)) 
print ("Features sorted by their score:" ) 
print (sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), x_train), reverse=True)  )

x_train.head()'''

In [ ]:
import sys
sys.path.insert(0, "/Users/schwalmdaniel/github/xgboost/python-package")
from xgboost import XGBClassifier
import lightgbm as lgb

#model = XGBClassifier()
#model.fit(x_train, y_train)
#print (model.score(x_train, y_train))

logit_model = XGBClassifier() #LogisticRegression()  
# Fit
logit_model = logit_model.fit(x_train, y_train,eval_metric='auc')  
# How accurate?
print (logit_model.score(x_train, y_train))  
#0.7874

# How does it perform on the test dataset?

# Predictions on the test dataset
predicted = pd.DataFrame(logit_model.predict(test2))  
# Probabilities on the test dataset
probs = pd.DataFrame(logit_model.predict_proba(test2))  
#prd[:,i] = np.round(probs,5)[:,1]
#print (metrics.accuracy_score(y_test, predicted)  )

print (probs.shape)

prd_1 = pd.DataFrame(probs)

import csv

submit = pd.concat([test['id'],prd_1],axis=1)

print (submit.columns.tolist)

submit = submit.drop(submit.columns[1], axis=1)
#probs.head()
submit.to_csv('donor1.csv',index=False,quoting=csv.QUOTE_NONNUMERIC)
submit.head()

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold, RepeatedKFold

import gc
import csv

cnt = 0
p_buf = []
n_splits = 2
n_repeats = 1
kf = RepeatedKFold(
    n_splits=n_splits, 
    n_repeats=n_repeats, 
    random_state=0)
auc_buf = []   

params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 12,
        'num_leaves': 31,
        'learning_rate': 0.025,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.85,
        'bagging_freq': 5,
        'verbose': 0,
        'num_threads': 4,
        'lambda_l2': 1.5,
        'min_gain_to_split': 0,
    }  

for train_index, valid_index in kf.split(x_train):
    print('Fold {}/{}'.format(cnt + 1, n_splits))
    
    model = lgb.train(
        params,
        lgb.Dataset(x_train.loc[train_index], y_train.loc[train_index], feature_name=x_train.columns.tolist()),
        num_boost_round=10000,
        valid_sets=[lgb.Dataset(x_train.loc[valid_index], y_train.loc[valid_index])],
        early_stopping_rounds=100,
        verbose_eval=100,
    )

    if cnt == 0:
        importance = model.feature_importance()
        model_fnames = model.feature_name()
        tuples = sorted(zip(model_fnames, importance), key=lambda x: x[1])[::-1]
        tuples = [x for x in tuples if x[1] > 0]
        print('Important features:')
        print(tuples[:200])

    p = model.predict(x_train.loc[valid_index], num_iteration=model.best_iteration)
    auc = roc_auc_score(y_train.loc[valid_index], p)

    print('{} AUC: {}'.format(cnt, auc))

    p = model.predict(test2, num_iteration=model.best_iteration)
    if len(p_buf) == 0:
        p_buf = np.array(p)
    else:
        p_buf += np.array(p)
    auc_buf.append(auc)

    cnt += 1
    if cnt > 0: # Comment this to run several folds
        break
    
    del model
    gc.collect

auc_mean = np.mean(auc_buf)
auc_std = np.std(auc_buf)
print('AUC = {:.6f} +/- {:.6f}'.format(auc_mean, auc_std))

preds = p_buf/cnt

subm = pd.DataFrame()
subm['id'] = test['id']
subm['project_is_approved'] = preds
subm.to_csv('talkingdata1.csv', index=False,quoting=csv.QUOTE_NONNUMERIC)
subm.head()


In [ ]:
train.hist('previous_projects_50')



In [ ]:
# Instantiate with a max depth of 3
tree_model = tree.DecisionTreeClassifier(max_depth=3)  
# Fit a decision tree
tree_model = tree_model.fit(x_train, y_train)  
# Training accuracy
tree_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(tree_model.predict(x_test))  
probs = pd.DataFrame(tree_model.predict_proba(x_test))

# Store metrics
tree_accuracy = metrics.accuracy_score(y_test, predicted)  
tree_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
tree_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
tree_classification_report = metrics.classification_report(y_test, predicted)  
tree_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
tree_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
tree_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

print (tree_accuracy)

# evaluate the model using 10-fold cross-validation
tree_cv_scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=3), x_test, y_test, scoring='precision', cv=10)

In [ ]:
# Instantiate
rf = RandomForestClassifier()  
# Fit
rf_model = rf.fit(x_train, y_train)  
# training accuracy 99.74%
rf_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(rf_model.predict(x_test))  
probs = pd.DataFrame(rf_model.predict_proba(x_test))

# Store metrics
rf_accuracy = metrics.accuracy_score(y_test, predicted)  
rf_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
rf_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
rf_classification_report = metrics.classification_report(y_test, predicted)  
rf_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
rf_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
rf_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

print (rf_accuracy)

# Evaluate the model using 10-fold cross-validation
#rf_cv_scores = cross_val_score(RandomForestClassifier(), x_test, y_test, scoring='precision', cv=10)  
#rf_cv_mean = np.mean(rf_cv_scores)  

In [ ]:
# Instantiate
svm_model = SVC(probability=True)  
# Fit
svm_model = svm_model.fit(x_train, y_train)  
# Accuracy
svm_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(svm_model.predict(x_test))  
probs = pd.DataFrame(svm_model.predict_proba(x_test))

# Store metrics
svm_accuracy = metrics.accuracy_score(y_test, predicted)  
svm_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
svm_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
svm_classification_report = metrics.classification_report(y_test, predicted)  
svm_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
svm_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
svm_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

print (svm_accuracy)

# Evaluate the model using 10-fold cross-validation
#svm_cv_scores = cross_val_score(SVC(probability=True), x_test, y_test, scoring='precision', cv=10)  
#svm_cv_mean = np.mean(svm_cv_scores)  

In [ ]:
# instantiate learning model (k = 3)
knn_model = KNeighborsClassifier(n_neighbors=3)  
# fit the model
knn_model.fit(x_train, y_train)  
# Accuracy
knn_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(knn_model.predict(x_test))  
probs = pd.DataFrame(knn_model.predict_proba(x_test))

# Store metrics
knn_accuracy = metrics.accuracy_score(y_test, predicted)  
knn_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
knn_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
knn_classification_report = metrics.classification_report(y_test, predicted)  
knn_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
knn_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
knn_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation
#knn_cv_scores = cross_val_score(KNeighborsClassifier(n_neighbors=3), x_test, y_test, scoring='precision', cv=10)  
#knn_cv_mean = np.mean(knn_cv_scores)  

In [ ]:
# Instantiate
bayes_model = GaussianNB()  
# Fit the model
bayes_model.fit(x_train, y_train)  
# Accuracy
bayes_model.score(x_train, y_train)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(bayes_model.predict(x_test))  
probs = pd.DataFrame(bayes_model.predict_proba(x_test))

# Store metrics
bayes_accuracy = metrics.accuracy_score(y_test, predicted)  
bayes_roc_auc = metrics.roc_auc_score(y_test, probs[1])  
bayes_confus_matrix = metrics.confusion_matrix(y_test, predicted)  
bayes_classification_report = metrics.classification_report(y_test, predicted)  
bayes_precision = metrics.precision_score(y_test, predicted, pos_label=1)  
bayes_recall = metrics.recall_score(y_test, predicted, pos_label=1)  
bayes_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

print (rf_accuracy)

# Evaluate the model using 10-fold cross-validation
#bayes_cv_scores = cross_val_score(KNeighborsClassifier(n_neighbors=3), x_test, y_test, scoring='precision', cv=10)  
#bayes_cv_mean = np.mean(bayes_cv_scores)  